In [1]:
    import jsonschema, aiohttp, fnmatch, uritemplate, dataclasses, builtins, jsonschema, matplotlib, collections, requests, networkx, pathlib, IPython, ruamel.yaml, \
    inspect, ujson as json, io, ibis, pandas, time, mimetypes, nbformat, aiofiles, aiohttp, nbformat, collections, functools
    from toolz.curried import *
    mime = compose(first, mimetypes.guess_type, str)            
    Path = type(pathlib.Path(''))
    def get_data(x): return x.data
    IPython.display.HTML("""<style>#notebook-container, .container {width: 100%;}</style>""")



In [3]:
    async def response(attr, url, **data):
        """If something has a path to IPython.display.JSON"""
        async with aiohttp.ClientSession() as session:
            async with session.get(url) as response: return await getattr(response, attr)()
    
    async def response_json(url): return await response('json', url)
    async def response_text(url): return await response('text', url)

In [4]:
    async def buffer(callable, path):
        async with aiofiles.open(path, mode='r') as file:
            if callable: return await callable(file)
            return file
        
    async def local_text(path): return await buffer(operator.methodcaller('read'), path)
    async def local_json(path): return json.loads(await local_text(path))

In [5]:
    class Validate:
        def __hash__(self): return hash(str(self.data))
        def __len__(self): return len(self.data)
        
    @dataclasses.dataclass(unsafe_hash=True)
    class Mime(Validate, collections.UserString):
        data: object = ''
        callable: callable = None
        async def __call__(self, object): 
            if self.callable:
                object = self.callable (object) 
                if inspect.isawaitable(object): object = await object
            return object

        def validate(self, object): 
            if isinstance(object, type(self)): return hash(object) == hash(self)
            return not self.data or self.data == mime(str(object))

In [6]:
    Text = Mime(callable=local_text)
    JSON = Mime(mime(' .json'), callable=local_json)

In [7]:
    class Schema(jsonschema.Draft4Validator, Validate):
        @property
        def data(self): return self.schema
        def validate(self, object):
            if isinstance(object, type(self)): return hash(object) == hash(self)
            try: return super().validate(object) or True
            except jsonschema.ValidationError: return False
        async def __call__(self, object): return object

In [8]:
    class Composition(networkx.DiGraph):
        async def paths(self, target, object):
            paths, mro = [], list(inspect.getmro(type(object)))
            while mro:
                klass, path_ = mro.pop(0), []
                if klass is builtins.object: break
                for method in (networkx.all_simple_paths, networkx.all_shortest_paths):
                    try: path_ += [object for object in method(self, klass, target) if len(object)>1]
                    except (networkx.NodeNotFound, networkx.NetworkXNoPath): ...
                paths += pipe(path_, map(take(2)), map(tuple), list, unique, list, partial(sorted, key=lambda x: (
                    len(x[1]) if isinstance(x[1], Validate) else 0, 
                    -(bool(getattr(x[1], 'callable', x[1])) if isinstance(x[1], Validate) else 10))))
            return paths

            
        async def advance(self, *targets, object):
            for target in targets:
                if target not in self: object = target(object); continue
                if not networkx.has_path(self, type(object), target): break
                
                callable, target = target, target.func if isinstance(target, functools.partial) else target
                prior, paths = object, await self.paths(target, object)
                if not paths: continue
                
                for this, next in paths:
                    if isinstance(next, Validate):
                        if next.validate(object): 
                            object, (this, next, *_) = await next(object), networkx.shortest_path(self, next, target)
                            break
                    else: break
                if next == target: next = callable    
                if next is builtins.object: continue
                    
                
                object = next() if object is None else next(object)                
                if inspect.isawaitable(object): object = await object
                    
                if next == target: continue
                elif isinstance(target, type):
                    if isinstance(type(object), target): continue
                else:
                    try:
                        if object == target: continue
                    except ValueError: ...
                        
                
                object = await self.advance(target, object=object)      
            return object
        
        async def __call__(self, target, *object, keys=True, **data, ):
            if not object: object = None,
            if not isiterable(target): target = target,
            value = [
                await self.advance(*target, object=object) for object in object
            ]
            if keys: return dict(zip(object, value))
            return value

In [9]:
    mimetypes.add_type('application/x-ipynb+json', '.ipynb')
    mimetypes.add_type('text/markdown', '.md')
    mimetypes.add_type('text/markdown', '.markdown')
    mimetypes.add_type('application/x-sqlite3', '.sqlite')
    mimetypes.add_type('application/x-yaml', '.yml')
    mimetypes.add_type('application/x-yaml', '.yaml')
    mimetypes.add_type('application/json', '.json')

In [10]:
    array = Schema({'type': 'array'})
    object_ = Schema({'type': 'object'})
    string = Schema({'type': 'string'})
    number = Schema({'type': 'number'})
    null = Schema({'type': 'null'})

In [44]:
    core = Composition()
    core.add_path([Path, Text, str])
    core.add_path([Path, JSON, object, object_, dict, json.dumps, str])
    core.add_path([Path, JSON, object, array, list, json.dumps])
    core.add_path([Path, JSON, object, string, str])
    core.add_path([Path, JSON, object, number, float, json.dumps])
    core.add_path([Path, Mime('application/x-ipynb+json', callable=local_json), nbformat.from_dict, nbformat.NotebookNode, dict])
    
    ado = core

In [45]:
    await ado.paths(object, Path('data.json'))

[(pathlib.PosixPath,
  Mime(data='application/json', callable=<function local_json at 0x1524fa8378>))]

In [47]:
    await ado((dict, dict.keys), Path('fastapi-openapi-context.ipynb'))

{PosixPath('fastapi-openapi-context.ipynb'): dict_keys(['cells', 'metadata', 'nbformat', 'nbformat_minor'])}

In [48]:
    await ado((nbformat.NotebookNode, juxt(type, dict.keys)), Path('fastapi-openapi-context.ipynb'))

{PosixPath('fastapi-openapi-context.ipynb'): (nbformat.notebooknode.NotebookNode,
  dict_keys(['cells', 'metadata', 'nbformat', 'nbformat_minor']))}

In [49]:
    await ado((str, get(slice(100))), Path('data.json'))

{PosixPath('data.json'): '{"index":{"0":"ZRRVgPxAbg","1":"qGdbMygKUl","2":"MFEpsjeg81","3":"Bc5QYEyyvO","4":"HmTeNtf2KE","5":"'}

In [50]:
    await ado.paths(dict, Path('data.json'))

[(pathlib.PosixPath,
  Mime(data='application/json', callable=<function local_json at 0x1524fa8378>)),
 (pathlib.PosixPath,
  Mime(data='application/x-ipynb+json', callable=<function local_json at 0x1524fa8378>))]

In [51]:
    await ado((dict, dict.keys), Path('data.json'))

{PosixPath('data.json'): dict_keys(['index', 'A', 'B', 'C', 'D'])}

In [52]:
    try: await ado(dict, Path('.travis.yml')); assert False, "Say it ain't so."
    except ValueError: ...

In [66]:
    yaml = Composition()
    yaml.add_path([Path, Mime('application/x-yaml', callable=local_text), io.StringIO, __import__('yaml').safe_load, object])
    ado = networkx.compose(ado, yaml)

In [67]:
    await ado((dict, dict.keys), Path('.travis.yml'))

{PosixPath('.travis.yml'): dict_keys(['language', 'python', 'install', 'script', 'deploy'])}

In [68]:
    await ado((str, get(slice(100))), Path('.travis.yml'))

{PosixPath('.travis.yml'): "language: python\npython:\n- '3.6'\n- 3.6-dev\ninstall:\n- python setup.py sdist bdist_wheel\n- pip instal"}

In [69]:
    displays = Composition()
    displays.add_path([str, IPython.display.HTML])
    displays.add_path([str, IPython.display.Markdown])
    displays.add_path([str, IPython.display.Code])
    displays.add_path([list, IPython.display.JSON])
    displays.add_path([dict, IPython.display.JSON])
    ado = networkx.compose(ado, displays)    

In [70]:
    await ado((IPython.display.Code, type), Path('.travis.yml'))

{PosixPath('.travis.yml'): IPython.lib.display.Code}

In [71]:
    pd = Composition()
    pd.add_path([Path, Mime('text/csv'), pandas.read_csv, pandas.DataFrame, pandas.DataFrame.transpose])
    pd.add_path([pandas.Series, pandas.DataFrame])
    pd.add_path([pandas.Series, dict])
    ado = networkx.compose(ado, pd)

In [72]:
    await ado(pandas.DataFrame.transpose, Path('data.csv'))

{PosixPath('data.csv'):                     0           1           2           3           4   \
 Unnamed: 0           0           1           2           3           4   
 index       ZRRVgPxAbg  qGdbMygKUl  MFEpsjeg81  Bc5QYEyyvO  HmTeNtf2KE   
 A             -1.13811    0.932169   -0.278003   -0.315296    -0.41495   
 B            -0.554226   0.0662085    0.804182   -0.558954   -0.107526   
 C            -0.303604    -1.12484    0.845789    0.703738   -0.243205   
 D            -0.304571     1.18461   -0.477547   -0.816047     0.85921   
 
                     5           6           7           8           9   ...  \
 Unnamed: 0           5           6           7           8           9  ...   
 index       64XpwGGKX4  FDGUaeSGrl  9bnZ4FYWx2  AwjEKKungG  NXaBII16Fj  ...   
 A             -1.29303   0.0666345    -1.32863     0.86664    0.414605  ...   
 B            -0.275852    -1.17587    0.703497    -1.45832    0.481333  ...   
 C             -2.29765    0.392907    -1.57246   